<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Tutorial de Pytorch 10:  Transfer Learning: VGG16**

'Transfer Learning' es un método de entrenamiento de redes neuronales que permite aprovechar el conocimiento adquirido por una red entrenada en un conjunto de datos para entrenar otra red en un conjunto de datos diferente. La idea es que la red pre-entrenada con un conjunto de datos (imágenes) variado y amplio, ha aprendido a extraer características útiles de las imágenes, por lo que se puede utilizar para extraer características de un conjunto de imágenes diferente que, presumiblemente, tiene características similares.

## **Datos**

Preparamos los datos igual que en el modelo anterior.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.nn import Dropout

# Establecemos semilla
torch.manual_seed(42)

# Definimos las transformaciones que deseas aplicar a las imágenes
transform = transforms.Compose([
    transforms.ToTensor(),           # Convierte las imágenes a tensores
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normaliza las imágenes para VGG16
])

# Ruta donde se encuentran las imágenes
data_dir = "./data/house_rooms"

# Creamos un objeto ImageFolder
dataset = ImageFolder(root=data_dir, transform=transform)

# Dividimos el conjunto de datos en entrenamiento (80%) y validación (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Definimos el tamaño del lote y creamos los DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## **Modelo**

Definimos el modelo usando las "features" de un modelo pre-entrenado, en este caso VGG16.

In [3]:
# Cargamos el modelo VGG16 preentrenado
model_conv = torchvision.models.vgg16()
vgg16 = torchvision.models.vgg16(weights='IMAGENET1K_V1')

# Congelamos los pesos de las últimas 7 capas convolutivas
for param in vgg16.features[:-7].parameters():
    param.requires_grad = False

# Obtenemos el número de características de la última capa
in_features = vgg16.classifier[0].in_features

# Creamos un nuevo clasificador
cls = nn.Sequential(
    nn.Linear(in_features, 256),  # 4096 -> 256
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 5),
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Creamos un nuevo modelo con las features de vgg16 y el clasificador personalizado
model = nn.Sequential(
    vgg16.features,
    nn.Flatten(),
    cls
).to(device)

In [4]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

## **Entrenamiento**

In [ ]:
# Definimos la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Validación del modelo
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation set: {100 * correct / total}%")